# Token Classificaiton (HuggingFace)
- NER, POS, Tagging, Chucking (which tokens belong to the same entity)

## 1. Load the data

thaisum dataset

In [1]:
#uncomment this if you are not using our department puffer
import os
os.environ['http_proxy']  = 'http://192.41.170.23:3128'
os.environ['https_proxy'] = 'http://192.41.170.23:3128'

In [2]:
import torch, torchdata, torchtext
from torch import nn
import torch.nn.functional as F
import random, math, time

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

#make our work comparable if restarted the kernel
SEED = 1234
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


cuda:0


In [3]:
from datasets import load_dataset

raw_datasets = load_dataset('thaisum')
raw_datasets

Found cached dataset thaisum (/root/.cache/huggingface/datasets/thaisum/thaisum/1.0.0/347b33c852af4d796e1224e00e15142d626608a9fa3e07ad6d19dfd8fcae5423)


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['title', 'body', 'summary', 'type', 'tags', 'url'],
        num_rows: 358868
    })
    validation: Dataset({
        features: ['title', 'body', 'summary', 'type', 'tags', 'url'],
        num_rows: 11000
    })
    test: Dataset({
        features: ['title', 'body', 'summary', 'type', 'tags', 'url'],
        num_rows: 11000
    })
})

In [4]:
raw_datasets['train'][0]

{'title': ' วิษณุ ยันโรดแม็ปเดิม ตอบไม่ถูกเวลาเลือกตั้ง ต้องรอ รธน.ประกาศใช้',
 'body': 'เมื่อวันที่ 6 ม.ค.60 ที่ทำเนียบรัฐบาล นายวิษณุ เครืองาม รองนายกรัฐมนตรี กล่าวถึงกรณี ที่ นายสุรชัย เลี้ยงบุญเลิศชัย รองประธานสภานิติบัญญัติแห่งชาติ (สนช.) ออกมาระบุว่า การเลือกตั้งจะถูกเลื่อนออกไปถึงปี 2561 ว่า ขอให้ไปสอบถามกับ สนช. แต่เชื่อว่าคงไม่กล้าพูดอีก เพราะทำให้คนเข้าใจผิด ซึ่งที่ สนช.พูดเนื่องจากผูกกับกฎหมายของกรรมการร่างรัฐธรรมนูญ(กรธ.) ตนจึงไม่ขอวิพากษ์วิจารณ์ แต่รัฐบาลยืนยันว่ายังเดินตามโรดแม็ป ซึ่งโรดแม็ปมองได้สองแบบ คือ มีลำดับขั้นตอนและการกำหนดช่วงเวลา โดยเริ่มต้นจากการประกาศใช้รัฐธรรมนูญ แต่ขณะนี้รัฐธรรมนูญยังไม่ประกาศใช้ จึงยังเริ่มนับหนึ่งไม่ถูก จากนั้นเข้าสู่ขั้นตอนการร่างกฎหมายประกอบร่างรัฐธรรมนูญหรือกฎหมายลูก ภายใน 240 วัน ก่อนจะส่งกลับให้ สนช.พิจารณา ภายใน 2 เดือน\xa0,นายวิษณุ กล่าวต่อว่า หากมีการแก้ไขก็จะมีการพิจารณาร่วมกับ กรธ.อีก 1 เดือน ก่อนนำขึ้นทูลเกล้าฯ ทรงลงพระปรมาภิไธย ภายใน 90 วัน และจะเข้าสู่การเลือกตั้งภายในระยะเวลา 5 เดือน ซึ่งทั้งหมดนี้คือโรดแม็ปที่ยังเป็นแบบเดิม

## 2. Tokenizer

The first component of the pipeline

- Splitting the input into words, subwords, or symbols (like punctuation) that are called tokens
- Mapping each token to an integer
- Adding additional inputs that may be useful to the model

To do this, we use the `AutoTokenizer` class and its `from_pretrained()` method. Using the checkpoint name of our model, it will automatically fetch the data associated with the model’s tokenizer and cache it (so it’s only downloaded the first time you run the code below).

Since the default checkpoint of the sentiment-analysis pipeline is `distilbert-base-uncased-finetuned-sst-2-english`, we run the following:

In [5]:
# !pip install sentencepiece

In [6]:
#To preprocess our data, we will need a tokenizer:
from transformers import CamembertTokenizer

checkpoint = 'airesearch/wangchanberta-base-att-spm-uncased'
tokenizer = CamembertTokenizer.from_pretrained(checkpoint)

In [7]:
##Tokenizer Example
raw_inputs = [
    "อกไก่ต้องมีคนหมัก คนหักต้องมีคนใหม่",
    "ไม่อยากติดตลก อยากติดเธอ ยืมตังห้าร้อยสิ",
    "ถ้าเขาจะรักยืนเฉยๆ เขาก็รัก"
]
inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="pt") #pt for pytorch
print(inputs)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'input_ids': tensor([[    5,    10,  1283,  1305, 19889,  8524,  2246,  1692, 19889,   126,
             6,     1,     1,     1],
        [    5, 10677,   266,  1776,  2169,   266,   128,    10,  4105,  2732,
          1850,  1772,   282,     6],
        [    5,   391,  1801,   235,  1015,  2658, 11663,   235,     6,     1,
             1,     1,     1,     1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0]])}


In [8]:
tokenizer.decode([5,   391,  1801,   235,  1015,  2658, 11663,   235,     6])

'<s> ถ้าเขาจะรักยืนเฉยๆ เขาก็รัก</s>'

In [9]:
def tokenize(examples):
    tokenizd_inputs = tokenizer(
        examples['summary'], 
        padding=True, 
        truncation=True, 
        return_tensors="pt"
        )
    return tokenizd_inputs

In [10]:
# tokenized_datasets = raw_datasets.map(
#     function = tokenize, batched=True, remove_columns=raw_datasets['train'].column_names,)

In [11]:
import pickle
# with open('tokenized_datasets.pickle', 'wb') as f:
#     pickle.dump(tokenized_datasets, f, protocol=pickle.HIGHEST_PROTOCOL)
with open('tokenized_datasets.pickle', 'rb') as f:
    tokenized_datasets = pickle.load(f)

In [12]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 358868
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 11000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 11000
    })
})

In [13]:
print(tokenized_datasets['train'][0]['input_ids'])

[5, 10, 10130, 10, 1669, 16740, 6982, 543, 10, 851, 10, 2380, 11, 712, 246, 3593, 216, 10, 3362, 1802, 10, 1121, 90, 19545, 10, 2417, 248, 1597, 16740, 96, 10, 4803, 11, 6487, 6, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [14]:
print(tokenizer.decode(tokenized_datasets['train'][0]['input_ids']))

<s> วิษณุ ยันโรดแม็ปตามขั้นตอนเดิม เชื่อ สนช.หยุดพูดขยับเลือกตั้ง ปัดวิจารณ์ ยึดตามกรอบเวลา ย้ําเริ่มนับโรดแม็ปเมื่อ รธน.ประกาศใช้</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><

In [15]:
print(tokenized_datasets['train'][0]['attention_mask'])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [16]:
from transformers import DataCollatorForLanguageModeling
#huggingface is very kind to make a data collator for each pipeline

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer)

### Loss

Great, that seems to work nicely! We can now write a custom loss function that takes the input sequence, the logits, and the key tokens we just selected as inputs. First we need to align the logits and inputs: the input sequence shifted by one to the right forms the labels, since the next token is the label for the current token. We can achieve this by starting the labels from the second token of the input sequence, since the model does not make a prediction for the first token anyway. Then we cut off the last logit, as we don’t have a label for the token that follows the full input sequence. With that we can compute the loss per sample and count the occurrences of all keywords in each sample. Finally, we calculate the weighted average over all samples using the occurrences as weights. Since we don’t want to throw away all the samples that have no keywords, we add 1 to the weights:

In [17]:
from torch.nn import CrossEntropyLoss
import torch

def keytoken_weighted_loss(inputs, logits, keytoken_ids, alpha=1.0):
    # Shift so that tokens < n predict n
    shift_labels = inputs[..., 1:].contiguous()
    shift_logits = logits[..., :-1, :].contiguous()
    # Calculate per-token loss
    loss_fct = CrossEntropyLoss(reduce=False) #change to reduction=None
    loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))
    # Resize and average loss per sample
    loss_per_sample = loss.view(shift_logits.size(0), shift_logits.size(1)).mean(axis=1)
    # Calculate and scale weighting
    weights = torch.stack([(inputs == kt).float() for kt in keytoken_ids]).sum(
        axis=[0, 2]
    )
    weights = alpha * (1.0 + weights)
    # Calculate weighted average
    weighted_loss = (loss_per_sample * weights).mean()
    return weighted_loss

## 3. Dataloader

In [18]:
# a small subset of the training and validation set, to enable faster training
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset  = tokenized_datasets["validation"].shuffle(seed=42).select(range(1000))  
small_test_dataset  = tokenized_datasets["test"].shuffle(seed=42).select(range(1000)) 

full_train_dataset = tokenized_datasets["train"]
full_eval_dataset = tokenized_datasets["validation"]
full_test_dataset = tokenized_datasets["test"]

Loading cached shuffled indices for dataset at /root/.cache/huggingface/datasets/thaisum/thaisum/1.0.0/347b33c852af4d796e1224e00e15142d626608a9fa3e07ad6d19dfd8fcae5423/cache-4074cb3982a74977.arrow
Loading cached shuffled indices for dataset at /root/.cache/huggingface/datasets/thaisum/thaisum/1.0.0/347b33c852af4d796e1224e00e15142d626608a9fa3e07ad6d19dfd8fcae5423/cache-4b6ef234d411218e.arrow
Loading cached shuffled indices for dataset at /root/.cache/huggingface/datasets/thaisum/thaisum/1.0.0/347b33c852af4d796e1224e00e15142d626608a9fa3e07ad6d19dfd8fcae5423/cache-d5d405c0a267d188.arrow


In [19]:
from torch.utils.data import DataLoader

batch_size = 16
train_loader = DataLoader(full_train_dataset,shuffle=True,
                          collate_fn=data_collator, batch_size=batch_size)
val_loader = DataLoader(full_eval_dataset,
                        collate_fn=data_collator, batch_size=batch_size)
test_loader = DataLoader(full_test_dataset,
                         collate_fn=data_collator, batch_size=batch_size)

In [20]:
weight_decay = 0.1


def get_grouped_params(model, no_decay=["bias", "LayerNorm.weight"]):
    params_with_wd, params_without_wd = [], []
    for n, p in model.named_parameters():
        if any(nd in n for nd in no_decay):
            params_without_wd.append(p)
        else:
            params_with_wd.append(p)
    return [
        {"params": params_with_wd, "weight_decay": weight_decay},
        {"params": params_without_wd, "weight_decay": 0.0},
    ]

Since we want to evaluate the model regularly on the validation set during training, let’s write a function for that as well. It just runs through the evaluation dataloader and gathers all the losses across processes:

In [21]:
def evaluate():
    model.eval()
    losses = []
    for step, batch in enumerate(val_loader):
        with torch.no_grad():
            outputs = model(batch["input_ids"].to(device), labels=batch["input_ids"].to(device))
            outputs.loss = outputs.loss.reshape(1)
        losses.append(accelerator.gather(outputs.loss))        
    loss = torch.mean(torch.cat(losses))
    try:
        perplexity = torch.exp(loss)
    except OverflowError:
        perplexity = float("inf")
    return loss.item(), perplexity.item()

With the `evaluate()` function we can report loss and perplexity at regular intervals. Next, we redefine our model to make sure we train from scratch again:

## 4.Model

the second component of the pipeline (after Tokenizer)

In [22]:
from transformers import AutoModelForMaskedLM #train with another linear layer 

checkpoint = 'airesearch/wangchanberta-base-att-spm-uncased'
model       = AutoModelForMaskedLM.from_pretrained(checkpoint)
model_size = sum(t.numel() for t in model.parameters())
print(f"wangchanberta size: {model_size/1000**2:.1f}M parameters")

wangchanberta size: 105.3M parameters


In [23]:
model = model.to(device)

We can then define our optimizer, using the function from before to split the parameters for weight decay:

In [24]:
from torch.optim import AdamW

#Adam with learning decay
optimizer = AdamW(model.parameters(),lr=2e-5)

### Accelerator

Now let’s prepare the model, optimizer, and dataloaders so we can start training:

In [25]:
# !pip install accelerate

In [26]:
from accelerate import Accelerator

accelerator = Accelerator()
model, optimzer, train_loader, eval_loader = accelerator.prepare(model,
                                                                 optimizer,
                                                                 train_loader,
                                                                 val_loader)

Now that we have sent our `train_dataloader` to `accelerator.prepare()`, we can use its length to compute the number of training steps. Remember that we should always do this after preparing the dataloader, as that method will change its length. We use a classic linear schedule from the learning rate to 0:

In [27]:
from transformers import get_scheduler

num_train_epochs = 1
num_update_steps_per_epoch = len(train_loader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    name="linear",
    optimizer=optimizer,
    num_warmup_steps=1_000,
    num_training_steps=num_training_steps,
)

### Repository

Lastly, to push our model to the Hub, we will need to create a `Repository` object in a working folder. First log in to the Hugging Face Hub, if you aren’t logged in already. We’ll determine the repository name from the model ID we want to give our model (feel free to replace the repo_name with your own choice; it just needs to contain your username, which is what the function `get_full_repo_name()` does):

In [28]:
# !pip install ipywidgets
# !apt install git

In [37]:
from huggingface_hub import notebook_login

notebook_login()

In [38]:
!git config --global credential.helper

Token is valid.
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credential helper as default.

git config --global credential.helper store

Read https://git-scm.com/book/en/v2/Git-Tools-Credential-Storage for more details.
Token has not been saved to git credential helper.
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [39]:
from huggingface_hub import Repository, get_full_repo_name

model_name = "wangchanberta-fintuned-mask-accelerate"
repo_name = get_full_repo_name(model_name)
repo_name

'guntsv/wangchanberta-fintuned-mask-accelerate'

In [40]:
# !apt install git-lfs

In [41]:
#sudo apt install git-lfs
#sudo brew install git-lfs
#go to git-lfs and download it
import os
os.environ["TOKENIZERS_PARALLELISM"] = 'true'

output_dir = "wangchanberta-fintuned-mask-accelerate"
repo = Repository(output_dir, clone_from=repo_name)

/root/nlp/09 - Story Generator using Hugging Face /wangchanberta-fintuned-mask-accelerate is already a clone of https://huggingface.co/guntsv/wangchanberta-fintuned-mask-accelerate. Make sure you pull the latest changes with `repo.git_pull()`.


We can now upload anything we save in `output_dir` by calling the `repo.push_to_hub()` method. This will help us upload the intermediate models at the end of each epoch.

## 5. Training

Before we train, let’s run a quick test to see if the evaluation function works properly:

In [42]:
model = model.to(device)

In [43]:
evaluate()

(12.0429105758667, 169890.703125)

Those are very high values for loss and perplexity, but that’s not surprising as we haven’t trained the model yet. With that, we have everything prepared to write the core part of the training script: the training loop. In the training loop we iterate over the dataloader and pass the batches to the model. With the logits, we can then evaluate our custom loss function. We scale the loss by the number of gradient accumulation steps so as not to create larger losses when aggregating more steps. Before we optimize, we also clip the gradients for better convergence. Finally, every few steps we evaluate the model on the evaluation set with our new `evaluate()` function:

In [ ]:
from tqdm.notebook import tqdm

gradient_accumulation_steps = 8
eval_steps = 5_000

model.train()
completed_steps = 0
for epoch in range(num_train_epochs):
    for step, batch in tqdm(
        enumerate(train_loader, start=1), total=num_training_steps
    ):
        logits = model(batch["input_ids"]).logits
        loss = keytoken_weighted_loss(batch["input_ids"], logits, keytoken_ids)
        if step % 100 == 0:
            accelerator.print(
                {
                    "steps": completed_steps,
                    "loss/train": loss.item() * gradient_accumulation_steps,
                }
            )
        loss = loss / gradient_accumulation_steps
        accelerator.backward(loss)
        if step % gradient_accumulation_steps == 0:
            accelerator.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()
            completed_steps += 1
        if (step % (eval_steps * gradient_accumulation_steps)) == 0:
            eval_loss, perplexity = evaluate()
            accelerator.print({"loss/eval": eval_loss, "perplexity": perplexity})
            model.train()
            accelerator.wait_for_everyone()
            unwrapped_model = accelerator.unwrap_model(model)
            unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)
            if accelerator.is_main_process:
                tokenizer.save_pretrained(output_dir)
                repo.push_to_hub(
                    commit_message=f"Training in progress step {step}", blocking=False
                )

## 6. Inference

Now is the moment of truth: let’s see how well the trained model actually works! We can see in the logs that the loss went down steadily, but to put the model to the test let’s take a look at how well it works on some prompts. To do that we’ll wrap the model in a text generation pipeline, and we’ll put it on the GPU for fast generations if there is one available.

In [49]:
from transformers import pipeline
checkpoint = 'guntsv/wangchanberta-fintuned-mask-accelerate'

pipe = pipeline("text-generation", max_length=100, pad_token_id=0, eos_token_id=0, model=checkpoint)

OSError: guntsv/wangchanberta-fintuned-mask-accelerate does not appear to have a file named config.json. Checkout 'https://huggingface.co/guntsv/wangchanberta-fintuned-mask-accelerate/main' for available files.

In [47]:
pipe("ประยุทธ์เตะ <mask>")